In [ ]:
!pip install langchain langchain-core faiss-cpu sentence-transformers pypdf langchain_huggingface -q

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from huggingface_hub import login
login(token="******")
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
# Step 1: Load your PDF file
pdf_path = "/content/ai_supply_chain_program.pdf"  # adjust path if needed
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} pages from the PDF.")

In [ ]:
# Step 2: Split text into chunks (optional, improves retrieval quality)
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # larger chunk
    chunk_overlap=100 # small overlap
)
docs = splitter.split_documents(documents)

In [ ]:
# Step 3: Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
# Step 4: Build FAISS vector store
vector_store = FAISS.from_documents(docs, embeddings)

In [ ]:
# Step 5: Create a retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# Step 6: Invoke the query
query = "who is Head of Program for the course AI-Driven Supply Chain Management"
results = retriever.invoke(query)

In [ ]:
# Print the retrived response
print(f"\n🔍 Query: {query}\n")
for i, doc in enumerate(results, 1):
    print(f"Result {i}:\n{doc.page_content}\n{'-'*80}")